In [1]:
library(tidyverse)
library(dplyr)
library(fgsea)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
res <- read.csv('../../results/postop/ruvg_runs/res5_17.csv')

res2 <- res %>% 
  dplyr::select(X, stat) %>% 
  na.omit() %>% 
  distinct() %>% 
  group_by(X) %>% 
  summarize(stat=mean(stat))

ranks <- deframe(res2)

pathways.hallmark <- gmtPathways("h.all.v2022.1.Hs.symbols.gmt")
pathways.hallmark %>%
    head() %>%
    lapply(head)

fgseaRes <- fgsea(pathways=pathways.hallmark, stats=ranks)

fgseaResTidy <- fgseaRes %>%
    as_tibble() %>%
    arrange(desc(NES))


#change leadingEdge data type to string
fgseaResTidy$leadingEdge <- as.character(fgseaResTidy$leadingEdge)



top10 = fgseaResTidy %>%
    arrange(padj) %>%
    head(20)
#replace '_' in pathway to ' '
top10$pathway <- gsub('_', ' ', top10$pathway)
#strip 'HALLMARK ' from pathway
top10$pathway <- gsub('HALLMARK ', '', top10$pathway)
#make pathway all lowercase
top10$pathway <- tolower(top10$pathway)

png('../../results/ruvg_runs/res5_17_hm.png', width=1440, height=1440)
ggplot(top10, aes(reorder(pathway, NES), NES)) +
  geom_col(fill = "#0066cc") +
  # scale_fill_manual(values="#0066cc") +
  coord_flip() +
  labs(x="Pathway", y="Normalized Enrichment Score",
       title="Hallmark Pathways", subtitle='Colon') + 
  theme_minimal(base_size=45)
dev.off()